## Add hexapod motions to the mount plots
### This runs multiple images

Craig Lage  24-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.efdUtils import calcNextDay

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
startDay = 20250609
endDay = 20250609

rmss = {}
dayObs = startDay
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['science', 'acq']:
            continue
        try:
            (mountErrors, mountData) = calculateMountErrors(record, client)
            rmss[record.id] = [mountErrors.camHexRms, mountErrors.m2HexRms]
            print(f"{record.id} succeeded!")
        except:
            print(f"{record.id} failed!")
            continue
        #if record.id > 2025060900090:
        #    break
    dayObs = calcNextDay(dayObs)

filename = f"/home/c/cslage/u/MTMount/mount_plots/Hex_RMS_{startDay}.pkl"
with open(filename, 'wb') as f:
    pkl.dump(rmss, f)


In [ ]:
dayObs = 20250609
filename = f"/home/c/cslage/u/MTMount/mount_plots/Hex_RMS_{dayObs}.pkl"
with open(filename, 'rb') as f:
    rmss = pkl.load(f)
print(len(rmss))
camHexRmss = []
m2HexRmss = []
expIds = []
for key in rmss.keys():
    [camHexRms, m2HexRms] = rmss[key]
    if (camHexRms < 1.0) and (m2HexRms < 1.0):
        camHexRmss.append(camHexRms)
        m2HexRmss.append(m2HexRms)
        expIds.append(key)

In [ ]:
plt.hist(camHexRmss, bins=20, range=(0,0.5))
plt.xlim(0, 0.5)

In [ ]:
instrument='LSSTCam'

fig, axs = plt.subplots(2, 2, figsize=(8,8))
plt.subplots_adjust(hspace=0.5)
pdfFig = plt.figure(figsize=(12, 10))
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Hex_RMS_Failures.pdf")
plt.subplots_adjust(hspace=0.5)
plotCounter = 1
for n, dayObs in enumerate([20250609, 20250619]):
    plotCounter = 0
    filename = f"/home/c/cslage/u/MTMount/mount_plots/Hex_RMS_{dayObs}.pkl"
    with open(filename, 'rb') as f:
        rmss = pkl.load(f)
    print(dayObs, len(rmss))
    camHexRmss = []
    m2HexRmss = []
    expIds = []
    for key in rmss.keys():
        [camHexRms, m2HexRms] = rmss[key]
        if (camHexRms < 1.0) and (m2HexRms < 1.0):
            camHexRmss.append(camHexRms)
            m2HexRmss.append(m2HexRms)
            expIds.append(key)
            if camHexRms > 0.25 and plotCounter < 10:
                where = "exposure.id=expId AND instrument='LSSTCam'"
                expRecords = butler.registry.queryDimensionRecords(
                            "exposure", where=where, bind={"expId": key}, datasets="raw"
                        )
                filteredExpRecords = set(expRecords)
                expRecord = filteredExpRecords.pop()
                (mountErrors, mountData) = calculateMountErrors(expRecord, client)
                pdfFig = plotMountErrors(mountData, mountErrors, figure=pdfFig) 
                pdfFig.suptitle(f"{key} - camHexRMS = {camHexRms:.2f}", fontsize=18)
                pdf.savefig(pdfFig)
                pdfFig.clf()
                plotCounter += 1
                
    axs[n][0].set_title(f"{dayObs}, CamHex", fontsize=18)
    axs[n][0].hist(camHexRmss, bins=20, range=(0,0.5))
    axs[n][0].set_xlim(0, 0.5)
    axs[n][0].set_xlabel("Image impact RMS (arcsec)")
    axs[n][1].set_title(f"{dayObs}, M2Hex", fontsize=18)
    axs[n][1].hist(m2HexRmss, bins=20, range=(0,0.5))
    axs[n][1].set_xlim(0, 0.5)
    axs[n][1].set_xlabel("Image impact RMS (arcsec)")
pdf.close()
fig.savefig("/home/c/cslage/u/MTMount/mount_plots/Hex_Impact_20250609_vs_20250619.png")